In [1]:
from io import BytesIO
import tempfile
import os
import base64
import hashlib
from typing import Callable

import numpy as np
import pandas as pd
from sklearn.decomposition import FactorAnalysis

import ipywidgets as widgets
from IPython.display import clear_output, display, HTML

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, FactorRange
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10
from bokeh.io import push_notebook

output_notebook(hide_banner=True)

## Correlation matrix data instructions
- Use `.xlsx` file.
- `.xlsx` file should only contain data, without any formatting.
- Specify __Y__ columns first in the Excel file, followed by __X__ columns.

In [2]:
def interactive_grouped_bar(corr_matrix):
    x_vars = list(corr_matrix.index)
    y_vars = list(corr_matrix.columns)

    factors = [(x, y) for x in x_vars for y in y_vars]

    corr_values = [corr_matrix.at[x, y] for x in x_vars for y in y_vars]

    source = ColumnDataSource(data=dict(
        x=factors,
        corr=corr_values,
    ))

    palette = Category10[10]
    color_map = factor_cmap('x', palette=palette, factors=y_vars, start=1, end=2)

    height_per_bar = 25
    min_height = 400
    plot_height = max(min_height, len(factors) * height_per_bar)
    plot_width = 600

    p = figure(
        y_range=FactorRange(*factors),
        height=plot_height,
        width=plot_width,
        title="Interactive Grouped Bar Chart: Correlation (X vs Y)",
        toolbar_location="above",
        tools="pan,wheel_zoom,box_zoom,reset,save"
    )


    p.hbar(y='x', right='corr', height=0.8, source=source, fill_color=color_map, line_color='black')

    p.x_range.start = -1
    p.x_range.end = 1
    p.xaxis.axis_label = "Correlation"
    p.yaxis.axis_label = "X variables grouped by Y variables"
    p.ygrid.grid_line_color = None
    p.yaxis.major_label_text_font_size = "9pt"

    hover = HoverTool(tooltips=[
        ("X variable, Y variable", "@x"),
        ("Correlation", "@corr{0.000}")
    ])
    p.add_tools(hover)

    show(p)

In [3]:
class DownloadButton(widgets.Button):
    def __init__(self, filename: str, contents, output_widget=None, **kwargs):
        super().__init__(**kwargs)
        self.filename = filename
        self.contents = contents
        self.output_widget = output_widget or widgets.Output()
        self.on_click(self.__on_click)

    def __on_click(self, b):
        contents: bytes = self.contents()
        b64 = base64.b64encode(contents)
        payload = b64.decode()
        digest = hashlib.md5(contents).hexdigest()  # bypass browser cache
        id = f'dl_{digest}'

        with self.output_widget:
            display(HTML(f"""
    <html>
    <body>
    <a id="{id}" download="{self.filename}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{payload}" download>
    </a>
    
    <script>
    (function download() {{
    document.getElementById('{id}').click();
    }})()
    </script>
    
    </body>
    </html>
    """))
        
def get_bytes_from_df(df):
    buffer = BytesIO()
    df.to_excel(buffer)
    buffer.seek(0)
    return buffer.read()

In [4]:
widget_raw_excel_file_upload = widgets.FileUpload(
    accept=".xlsx",
    multiple=False,
    description='Upload .xlsx file'    
)
widget_num_y_cols = widgets.BoundedIntText(
    value=3,
    min=1,
    max=100,
    step=1,
    description="Number of Y columns:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)
widget_compute_corr_matrix_button = widgets.Button(
    description="Compute correlation matrix",
    layout=widgets.Layout(width='200px')
)
widget_corr_matrix_output = widgets.Output()

corr_matrix = None
corr_sorted_matrix = None
    
widget_downlaod_corr_matrix_button = DownloadButton(
    filename="corr_matrix.xlsx",
    contents=lambda: get_bytes_from_df(corr_matrix),
    output_widget=widget_corr_matrix_output,
    description='Download correlation matrix',
    disabled=True,
    layout=widgets.Layout(width='250px')
)
widget_downlaod_corr_sorted_matrix_button = DownloadButton(
    filename="corr_sorted_matrix.xlsx",
    contents=lambda: get_bytes_from_df(corr_sorted_matrix),
    output_widget=widget_corr_matrix_output,
    description='Download sorted correlation matrix',
    disabled=True,
    layout=widgets.Layout(width='250px')
)

def compute_correlation_matrix(change):
    global corr_matrix
    global corr_sorted_matrix
    
    with widget_corr_matrix_output:
        clear_output()
        
        if not widget_raw_excel_file_upload.value:
            print("Please upload an Excel file.")
            return

        # Read Excel file
        raw_excel_data = widget_raw_excel_file_upload.value[0]['content']
        try:
            df_raw = pd.read_excel(BytesIO(raw_excel_data), engine='openpyxl')
            print("DataFrame (Excel file) loaded successfully (showing 10 rows):")
            display(df_raw.head(10))
        except Exception as e:
            print(f"Error reading Excel file: {e}")
            return

        # Split Y and X columns
        num_y_cols = widget_num_y_cols.value
        df_y = df_raw.iloc[:, :num_y_cols]
        df_x = df_raw.iloc[:, num_y_cols:]

        corr_matrix = df_x.join(df_y).corr().loc[df_x.columns, df_y.columns]

        print('Correlation matrix')
        display(corr_matrix.round(3))

        print('Correlation matrix sorted by sum of absolute correlations to Y columns:')
        corr_sums = corr_matrix.abs().sum(axis=1)
        corr_sorted_matrix = corr_matrix.loc[corr_sums.sort_values(ascending=False).index]
        display(corr_sorted_matrix.round(3))

        widget_downlaod_corr_matrix_button.disabled = False
        widget_downlaod_corr_sorted_matrix_button.disabled = False

        interactive_grouped_bar(corr_matrix)
        
widget_compute_corr_matrix_button.on_click(compute_correlation_matrix)

display(
    widgets.VBox(
        [
            widget_raw_excel_file_upload,
            widget_num_y_cols,
            widget_compute_corr_matrix_button,
            widget_downlaod_corr_matrix_button,
            widget_downlaod_corr_sorted_matrix_button,
            widget_corr_matrix_output,
        ]
    )
)

## Factor analysis data instructions
- Use `.xlsx` file.
- `.xlsx` file should only contain data, without any formatting.
- Specify __Y__ columns first in the Excel file, followed by __X__ columns.

In [6]:
widget2_raw_excel_file_upload = widgets.FileUpload(
    accept=".xlsx",
    multiple=False,
    description='Upload .xlsx file'    
)
widget2_num_y_cols = widgets.BoundedIntText(
    value=3,
    min=1,
    max=100,
    step=1,
    description="Number of Y columns:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)
widget2_num_output_dims = widgets.BoundedIntText(
    value=3,
    min=1,
    max=100,
    step=1,
    description="Number of dimensions to reduce X to:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)
widget2_compute_factor_analysis_button = widgets.Button(
    description="Compute factor analysis",
    layout=widgets.Layout(width='200px')
)
widget2_factor_analysis_output = widgets.Output()

df_factors = None
df_loadings = None

widget2_downlaod_factor_analysis_matrix_button = DownloadButton(
    filename="factor_analysis_matrix.xlsx",
    contents=lambda: get_bytes_from_df(df_factors),
    output_widget=widget2_factor_analysis_output,
    description='Download factor analysis matrix',
    disabled=True,
    layout=widgets.Layout(width='250px')
)
widget2_downlaod_loadings_matrix_button = DownloadButton(
    filename="loadings_matrix.xlsx",
    contents=lambda: get_bytes_from_df(df_loadings),
    output_widget=widget2_factor_analysis_output,
    description='Download loadings matrix',
    disabled=True,
    layout=widgets.Layout(width='250px')
)
def compute_factor_analysis(change):
    global df_factors
    global df_loadings
    
    with widget2_factor_analysis_output:
        clear_output()
        
        if not widget2_raw_excel_file_upload.value:
            print("Please upload an Excel file.")
            return

        # Read Excel file
        raw_excel_data = widget2_raw_excel_file_upload.value[0]['content']
        try:
            df_raw = pd.read_excel(BytesIO(raw_excel_data), engine='openpyxl')
            print("DataFrame (Excel file) loaded successfully (showing 10 rows):")
            display(df_raw.head(10))
        except Exception as e:
            print(f"Error reading Excel file: {e}")
            return

        # Split Y and X columns
        num_y_cols = widget2_num_y_cols.value
        df_y = df_raw.iloc[:, :num_y_cols]
        df_x = df_raw.iloc[:, num_y_cols:]

        new_x_dim = widget2_num_output_dims.value

        fa = FactorAnalysis(n_components=new_x_dim, random_state=0)
        x_factors = fa.fit_transform(df_x)

        df_factors = pd.DataFrame(
            x_factors,
            columns=[f'Factor {i+1}' for i in range(x_factors.shape[1])],
            index=df_x.index
        )
        
        df_loadings = pd.DataFrame(
            fa.components_.T,
            index=df_x.columns,
            columns=[f'Factor {i+1}' for i in range(fa.components_.shape[0])]
        )

        print('New features')
        display(df_factors.round(3))
        
        print('Relationship between original variables and latent factors:')
        display(df_loadings.round(3))

        widget2_downlaod_factor_analysis_matrix_button.disabled = False
        widget2_downlaod_loadings_matrix_button.disabled = False
        
widget2_compute_factor_analysis_button.on_click(compute_factor_analysis)

display(
    widgets.VBox(
        [
            widget2_raw_excel_file_upload,
            widget2_num_y_cols,
            widget2_num_output_dims,
            widget2_compute_factor_analysis_button,
            widget2_downlaod_factor_analysis_matrix_button,
            widget2_downlaod_loadings_matrix_button,
            widget2_factor_analysis_output,
        ]
    )
)